<font face="Times New Roman" size=5>
<div dir=rtl align="center">
<font face="Times New Roman" size=5>
In The Name of God
</font>
<br>
<img src="https://logoyar.com/content/wp-content/uploads/2021/04/sharif-university-logo.png" alt="University Logo" width="150" height="150">
<br>
<font face="Times New Roman" size=4 align=center>
Sharif University of Technology - Department of Electrical Engineering
</font>
<br>
<font color="#008080" size=6>
Deep Generative Models
</font>

<hr/>
<font color="#800080" size=5>
Assignment 4: Energy-based Models
<br>
</font>
<font size=5>
Instructor: Dr. S. Amini
<br>
</font>
<font size=4>
Fall 2024
<br>
</font>
<font face="Times New Roman" size=4>
Deadline: Month day at 23:55
</font>
<hr>
</div></font>

This homework helps you implement a Energy-based Model (EBM).  
For references please refer to the original [Implicit Generation and Generalization in Energy-Based Models](https://arxiv.org/abs/1903.08689).

In this homework you need to complete the notebook and run all the cells.
We have specified the parts to be completed with `TODO` tags inside the code blocks.

**NOTES**:
* Feel free to change any part of notebook, but remeber to use comments for clarification.
* You can get **complete score** for this notebook if your codes work properly and without errors.
* This notebook is tested with *Google Colab*.

In [3]:
name = "AmirHossein Naghdi"
studentId = "400102169"

# Energy-based Models (EBMs) (80 pts)

## Introduction

In this notebook, we will look at energy-based deep learning models, and focus on their application as generative models.

First, let's import our libraries below.

In [4]:
# Standard libraries
import numpy as np
import random

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

# Torchvision
import torchvision
from torchvision.datasets import MNIST
from torchvision import transforms
from torchvision.utils import make_grid

device = torch.device('cuda') if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

## Energy Models

As we have seen in the class, Energy-based models tend to predict probability density distribution. For this purpose we need to learn given probability:



$p_{\theta }( x_{train}) =\frac{e^{f_{\theta }( x)}}{Z( \theta )}$



Here $p_{\theta }( x_{train})$ stands for probability density of our training data distribution. We know that calculating the partition function ($Z( \theta )$) is interactable. For this purpose we will use contrastive divergence algorithm.

### Contrastive Divergence

We train generative models usually by maximum likelihood estimation. But the exact likelihood of a point cannot be determined due to intractable normalization constant $Z(\theta)$, we need to train energy-based models slightly different. We can re-write our maximum likelihood objective where we maximize the probability of $\mathbf{x}_{\text{train}}$ compared to a randomly sampled data point of our model:

$$
\begin{split}
    \nabla_{\theta}\mathcal{L}_{\text{MLE}}(\mathbf{\theta}) & = \nabla_{\theta}\log p_{\theta}(\mathbf{x})\\[5pt]
    & \approx  \nabla_{\theta}F_{\theta}(\mathbf{x_{\text{train}}}) - \nabla_{\theta}F_{\theta}(\mathbf{x_{\text{sample}}})
\end{split}
$$

Note that the loss is still an objective we want to minimize. Thus, we try to minimize the energy for data points from the dataset, while maximizing the energy for randomly sampled data points from our model. The trick for sampling is that we approximate $Z(\theta)$ by a single Monte-Carlo sample. This gives us the exact same objective as written above.

Visually, the objective is shown below (figure - [Stefano Ermon and Aditya Grover](https://deepgenerativemodels.github.io/assets/slides/cs236_lecture11.pdf)):


<center width="100%"><img src="https://uvadlc-notebooks.readthedocs.io/en/latest/_images/contrastive_divergence.svg" width="90%"></center>


The point on the right, called "correct answer", represents a data point from the dataset (i.e. $x_{\text{train}}$), and the left point, "wrong answer", a sample from our model (i.e. $x_{\text{sample}}$). Thus, we try to "pull up" the probability of the data points in the dataset, while "pushing down" randomly sampled points.

### Dataset

First, we load the MNIST dataset below. Note that we need to normalize the images between -1 and 1 instead of mean 0 and std 1 because during sampling, we have to limit the input space. Scaling between -1 and 1 makes it easier to implement it.

In [5]:
############ TODO #############:
BATCH_SIZE = 64

transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors with values in [0, 1]
    transforms.Normalize((0.,), (1.,))  # Normalize to mean=0., std=1. to scale to [-1, 1]
])
###############################

train_set = MNIST(root='./', train=True, transform=transform, download=True)
test_set = MNIST(root='./', train=False, transform=transform, download=True)

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True,  drop_last=True, pin_memory=True)
test_loader  = DataLoader(test_set,  batch_size=BATCH_SIZE, shuffle=False, drop_last=False)


### CNN Model

First, we implement our CNN model. The MNIST images are 28x28, hence a small model is enough. But you are free to use pre-trained Convolution Networks (such as Resnet, etc.)

Here we use a smooth activation function like Swish instead of ReLU in the energy model. This is because gradients should not be sparse.

In [6]:
class Swish(nn.Module):

    def forward(self, x):
        return x * torch.sigmoid(x)

class CNNModel(nn.Module):

    def __init__(self, hidden_features=32, out_dim=1):
        super(CNNModel, self).__init__()
        ############ TODO #############:
        # Define hidden layers (Conv) in and out dimensions
        hidden_dim1 = hidden_features // 2
        hidden_dim2 = hidden_features
        hidden_dim3 = hidden_features * 2
        ################################

        # Series of convolutions and Swish activation functions
        # your're free to implement any CNN Network. You can also use pretrained networks (like ResNet, etc.) instead.
        ############ TODO #############:
        self.cnn_net = nn.Sequential(
                nn.Conv2d(1, hidden_dim1, kernel_size=5, stride=2, padding=4), # [16x16] - Larger padding to get 32x32 image
                Swish(),
                nn.Conv2d(hidden_dim1, hidden_dim2, kernel_size=3, stride=2, padding=1), #  [8x8]
                Swish(),
                nn.Conv2d(hidden_dim2, hidden_dim3, kernel_size=3, stride=2, padding=1), # [4x4]
                Swish(),
                nn.Conv2d(hidden_dim3, hidden_dim3, kernel_size=3, stride=2, padding=1), # [2x2]
                Swish(),
                nn.Flatten(),
                nn.Linear(hidden_dim3*4, hidden_dim3),
                Swish(),
                nn.Linear(hidden_dim3, out_dim)
        )
        ################################

    def forward(self, x):
        # Feed the input to the network and return the output
        ############ TODO #############:
        # Feed the input through the convolutional network
        x = self.cnn_net(x).squeeze(dim=-1)
        return x
        ################################

### Sampling buffer

In this part, we construct our sampler. To use the contrastive divergence objective, we need to generate samples during training. The simple way to sampling is to use simple MCMC algorithm, but in practice it costs too much. So here we use *Langevin MCMC* that uses gradient for better sampling.

<center width="100%"><img src="https://uvadlc-notebooks.readthedocs.io/en/latest/_images/sampling.svg" width="90%"></center>

Below, we implement the sampling class. The function `sample_new_exmps` returns a new batch of "fake" images . These images are generated using MCMC. For more efficiency, each time we only generate 5% of sample size and other 95% are randomly picked from our buffer. We perform MCMC for 80 iterations to improve the image quality and come closer to samples from $p_{\theta}(\mathbf{x})$. In the function `generate_samples`, we implemented the MCMC for images. Note that the hyperparameters in this code are for MNIST dataset.

In [7]:
class Sampler:

    def __init__(self, model, img_shape, sample_size, max_len=8192):
        """
        Inputs:
            model - Neural network to use for modeling E_theta
            img_shape - Shape of the images to model
            sample_size - Batch size of the samples
            max_len - Maximum number of data points to keep in the buffer
        """
        super().__init__()
        self.model = model
        self.img_shape = img_shape
        self.sample_size = sample_size
        self.max_len = max_len
        self.examples = [(torch.rand((1,) + img_shape) * 2 - 1) for _ in range(self.sample_size)]

    def sample_new_exmps(self, steps=60, step_size=10):
        """
        Function for getting a new batch of "fake" images.
        Inputs:
            steps - Number of iterations in the MCMC algorithm
            step_size - Learning rate nu in the algorithm above
        """
        # Choose 95% of the batch from the buffer, 5% generate from scratch
        # Only generate 5% of sample size and use other 95% from buffer (examples)
        # You can generate samples the same way they're generated in "examples"
        # Then randomly choose 95% of sample size from examples and add newly generated one to it
        ############ TODO #############:
        # Generate 5% new samples
        # Choose 95% of the batch from the buffer, 5% generate from scratch
        n_new = np.random.binomial(self.sample_size, 0.05)
        rand_imgs = torch.rand((n_new,) + self.img_shape) * 2 - 1
        old_imgs = torch.cat(random.choices(self.examples, k=self.sample_size-n_new), dim=0)
        inp_imgs = torch.cat([rand_imgs, old_imgs], dim=0).detach().to(device)
        ################################

        # Perform MCMC sampling
        inp_imgs = Sampler.generate_samples(self.model, inp_imgs, steps=steps, step_size=step_size)

        # Add new images to the buffer and remove old ones if needed
        self.examples = list(inp_imgs.to(torch.device("cpu")).chunk(self.sample_size, dim=0)) + self.examples
        self.examples = self.examples[:self.max_len]
        return inp_imgs

    @staticmethod
    def generate_samples(model, newly_imgs, steps=60, step_size=10, return_img_per_step=False):
        """
        Function for sampling images for a given model.
        Inputs:
            model - Neural network to use for modeling E_theta
            newly_imgs - Images to start from for sampling. If you want to generate new images.
            steps - Number of iterations in the Langevin MCMC algorithm.
            step_size - Learning rate nu in the algorithm above
            return_img_per_step - If True, we return the sample at every iteration of the MCMC
        """
        # Before MCMC: set model parameters to "required_grad=False"
        # because we are only interested in the gradients of the input.
        is_training = model.training
        model.eval()
        for p in model.parameters():
            p.requires_grad = False
        newly_imgs.requires_grad = True

        # Enable gradient calculation if not already the case
        had_gradients_enabled = torch.is_grad_enabled()
        torch.set_grad_enabled(True)

        # we use a buffer tensor in which we generate noise each loop iteration.
        # more efficient than creating a new tensor every iteration.
        # generate random tensor using image shape
        ############ TODO #############:
        # print(f'{newly_imgs.shape=}')
        noise = torch.randn(newly_imgs.shape, device=newly_imgs.device)
        ################################

        # List for storing generations at each step (for later analysis)
        imgs_per_step = []

        # Loop over K (steps)
        for _ in range(steps):
            # You can use tensor functions to complete the code. eg. tensor.add, tensor.grad

            # Part 1:
            # Sample noise tensor data from a normal dist (mu=0, std=0.005).
            # Add the noise to the new image
            # Clamp it between -1 and 1
            ############ TODO #############:
            noise.normal_(0, 0.005)
            newly_imgs.data.add_(noise.data)
            newly_imgs.data.clamp_(min=-1.0, max=1.0)
            ################################

            # Part 2:
            # Feed new image to the model (**model output sign is important)
            # Use backward on sum of output to calculate gradient
            # Clamp tensor gradient between -0.05 and 0.05 (preventing too high gradients)
            ############ TODO #############:
            output = -model(newly_imgs)
            output.sum().backward()
            newly_imgs.grad.data.clamp_(-0.03, 0.03) # For stabilizing and preventing too high gradients
            ################################

            # Part 3:
            # Add grad to the image with step_size (like the given algorithm)
            ############ TODO #############:
            newly_imgs.data.add_(-step_size * newly_imgs.grad.data)
            ################################

            newly_imgs.grad.detach_()
            newly_imgs.grad.zero_()
            newly_imgs.data.clamp_(min=-1.0, max=1.0)

            if return_img_per_step:
                imgs_per_step.append(newly_imgs.clone().detach())

        # Reactivate gradients for parameters for training
        for p in model.parameters():
            p.requires_grad = True
        model.train(is_training)

        # Reset gradient calculation to setting before this function
        torch.set_grad_enabled(had_gradients_enabled)

        if return_img_per_step:
            return torch.stack(imgs_per_step, dim=0)
        else:
            return newly_imgs

### Training algorithm

Below is shown a summary of the full training algorithm of an energy model:

<center width="100%"><img src="https://uvadlc-notebooks.readthedocs.io/en/latest/_images/training_algorithm.svg" width="80%"></center>

Given algorithm, for each batch of data we're going to, first, add a small noise to image, then we sample a fake image using `sample_new_example`. After this we feed these images to our `Model`. At the end we use our `ebm_loss` to calculate loss and backward to complete our trainig.

For evaluation we use almost the same structure as our training, but with a different loss.

After training phase we're going to use `generate_image` to sample from our model (generate images).

In [8]:
# Implement the given algorithm
# In order to complete code you have to add the small noise to image
# Sample a fake image
# Feed both real image and fake image to the network
# And use the ebm loss
############ TODO #############:
def train_one_epoch(model, train_loader, optimizer, sampler, alpha):
    """
    Input:
        model: implemented CNN
        train_loader: train Dataloader
        optimizer: Optimizer to optimize model
        sampler: Sampler to generate samples
        alpha: hyperparameter for loss
    Returns:
        float: Epoch loss
    """
    model.train()
    total_loss = 0
    total_loss_cd = 0
    total_loss_rg = 0

    for img, _ in train_loader:
        img = img.to(device)
        small_noise = torch.randn_like(img) * 0.005
        img.add_(small_noise).clamp_(min=-1.0, max=1.0)

        # Generate fake images using the sampler
        fake_imgs = sampler.sample_new_exmps(steps=60, step_size=10)

        # Feed both real and fake images to the model
        # print(f'{img.shape=}')
        # print(f'{fake_imgs.shape=}')
        real_out = model(img)
        fake_out = model(fake_imgs)

        # Compute the contrastive divergence loss: E(G(z)) - E(x)
        loss_cd = fake_out.mean() - real_out.mean()
        total_loss_cd += loss_cd.item()

        # Compute the regularization loss: E(x)^2 + E(G(z))^2
        loss_rg = (real_out ** 2 + fake_out ** 2).mean()
        total_loss_rg += loss_rg.item()

        loss = loss_cd + alpha * loss_rg
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    mean_loss = total_loss / len(train_loader)
    mean_loss_cd = total_loss_cd / len(train_loader)
    mean_loss_rg = total_loss_rg / len(train_loader)

    return mean_loss, mean_loss_cd, mean_loss_rg
################################

def test_one_epoch(model, test_loader):
    # For validating, we use the contrastive divergence between purely random images and unseen examples
    total_cdiv = 0

    for imgs, _ in train_loader:
        imgs = imgs.to(device)
        fake_imgs = torch.rand_like(imgs) * 2 - 1

        real_out = model(imgs)
        fake_out = model(fake_imgs)

        cdiv = fake_out.mean() - real_out.mean()
        total_cdiv += cdiv.item()

    return total_cdiv / len(test_loader)

# Implement sample generator
# Create a random noise using image shape and sample size
# Use generate_samples to get samples
############ TODO #############:
def generate_image(model, sample_size, steps=256, step_size=16):
    model.eval()
    start_imgs = torch.randn((sample_size,) + img_shape).to(device)  # Start with random noise
    start_imgs = start_imgs * 2 - 1  # Scale noise to [-1, 1]
    torch.set_grad_enabled(True)  # Tracking gradients for sampling necessary
    imgs_per_step = Sampler.generate_samples(model, start_imgs, steps=steps, step_size=step_size, return_img_per_step=True)
    torch.set_grad_enabled(False)
    return imgs_per_step
################################

### Loss

Here we define our Energy-based Model loss. EBM loss consist of two different losses.


1.   Contrastive divergence using our energy model $f_{\theta}$
2.   Regularization term is additional training trick to ensure that the output values are in a reasonable range. Without this regularization loss, the output values will fluctuate in a very large range

`alpha` is a hyperparameter for regularization term ($\alpha<1$)

In [9]:
# Implement the given EBM loss
############ TODO #############:
def ebm_loss(real, fake, alpha):
    loss_cd = real.mean() - fake.mean()
    loss_rg = (fake.mean() ** 2)
    loss = loss_cd + loss_rg * alpha
    return loss
################################

### Instantiation

Instantiat your **Model**, **Sampler** and **Optimizer**.

In [10]:
lr = 1e-4
alpha = 0.1
img_shape = (1, 28, 28)

model = CNNModel().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
sampler = Sampler(model, img_shape, BATCH_SIZE)

### Train Loop

Train your model and print loss for each epoch.

In [ ]:
train_loss = []
epoch = 30
for i in range(epoch):
  train_loss.append(train_one_epoch(model, train_loader, optimizer, sampler, alpha))
  print(f'Epoch {i+1}: Train Loss = {train_loss[-1]}')

Epoch 1: Train Loss = (-0.0009895012170337467, -0.0010154986055164073, 0.000259973886574253)
Epoch 2: Train Loss = (-2.3348821038587284e-05, -3.0398695741041106e-05, 7.049875031017277e-05)


### Test
Test your Model using Test Dataloader

In [ ]:
test_one_epoch(model, test_loader)

### Generate Image

Here you have to use `generate_image` to sample from model and visualize it.

In [ ]:
############ TODO #############:
images_per_step = generate_image(model, sample_size=4, step_size=12)
images_per_step = images_per_step.cpu()
################################

for i in range(images_per_step.shape[1]):
    imgs_to_plot = images_per_step[0:256:32,i]
    grid = torchvision.utils.make_grid(imgs_to_plot, nrow=imgs_to_plot.shape[0], normalize=True, pad_value=0.5, padding=2)
    grid = grid.permute(1, 2, 0)
    plt.figure(figsize=(8,8))
    plt.imshow(grid.clone().cpu())
    plt.xlabel("Generation iteration")
    plt.show()

# Evaluation (20 pts)

Now lets evaluate the model more precisely.

In [1]:
import numpy as np
from scipy.spatial import cKDTree

def compute_identifiability(orig_data, synth_data):
    """
    Args:
        orig_data: original data
        synth_data: synthetically generated data

    Returns:
        identifiability_value: Identifiability value
    """
    def compute_entropy(labels):
        """
        Compute entropy of the given labels.

        Args:
            labels: input labels

        Returns:
            entropy: computed entropy
        """
        values, counts = np.unique(labels, return_counts=True)
        probabilities = counts / counts.sum()
        entropy = -np.sum(probabilities * np.log2(probabilities + 1e-16))
        return entropy

    orig_data = np.asarray(orig_data)
    synth_data = np.asarray(synth_data)

    no, x_dim = np.shape(orig_data)

    W = np.zeros([x_dim,])

    # Compute weights using entropy of each dimension
    for i in range(x_dim):
        W[i] = compute_entropy(orig_data[:, i])

    # Normalize original and synthetic data
    orig_data_hat = orig_data / (W + 1e-16)
    synth_data_hat = synth_data / (W + 1e-16)

    eps = 1e-16  # Small epsilon to avoid division by zero.

    # Compute the distances to the two nearest neighbors for each data point in the original data
    orig_tree = cKDTree(orig_data_hat)
    d_orig, idx_orig = orig_tree.query(orig_data_hat, k=3)

    # Compute the distances from each point in the original data to its nearest neighbor in the synthetic data
    synth_tree = cKDTree(synth_data_hat)
    d_synth, idx_synth = synth_tree.query(orig_data_hat, k=1)

    # Compute the identifiability metric
    '''
    Calculate the difference between the nearest synthetic neighbor distance and
    the second nearest original neighbor distance for each data point and
    count how many times this difference is negative.
    Compute the identifiability value as the fraction of times this occurs.
    '''
    second_nearest_orig_distances = d_orig[:, 2]
    nearest_synth_distances = d_synth

    negative_differences = (nearest_synth_distances - second_nearest_orig_distances) < 0
    identifiability_value = np.mean(negative_differences)

    return identifiability_value


In [ ]:
origin_data = []
for img, _ in test_loader:
    origin_data.append(img.numpy())

origin_data = np.concatenate(origin_data, axis=0)

origin_data = origin_data.reshape(-1, 28*28)

synth_data = []
for _ in range(len(test_loader)):
    img = generate_image(model, BATCH_SIZE).squeeze().cpu().numpy()
    synth_data.append(img)

synth_data = np.concatenate(synth_data, axis=0)

synth_data = synth_data.reshape(-1, 28*28)


identifiability_value = compute_identifiability(origin_data, synth_data)
print(identifiability_value)